In [1]:
!pip install -q kss
%tensorflow_version 2.x

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
   
# PyDrive Authentication
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
folder_id = '1wV4k91wu5NVNuC-KHSBv2xxqpFSVd0gt'
   
def ListFolder(parent):
    filelist=[]
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
    for f in file_list:
        if f['mimeType']=='application/vnd.google-apps.folder': # if folder
            filelist.append({"id":f['id'],"title":f['title'],"list":ListFolder(f['id'])})
        else:
            filelist.append({"title":f['title'],"id":f['id']})
    return filelist
   
   
file_lists = ListFolder(folder_id)

In [4]:
file_lists_from_drive = ListFolder(folder_id)
   
# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/my_sample_data')
try:
    os.makedirs(local_download_path)
except: pass
   
for file in file_lists_from_drive:
    if file['title'] != 'CM.txt':
        continue
    print('title: %s, id: %s' % (file['title'], file['id']))
    fname = os.path.join(local_download_path, file['title'])
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': file['id']})
    f_.GetContentFile(fname)
    print(fname)
    file_lists.append(fname)
   
# print file lists
print(file_lists)

title: CM.txt, id: 1QHq6p67jpE56AI5c1HPhJifTpSYaB9Gl
downloading to /root/my_sample_data/CM.txt
/root/my_sample_data/CM.txt
[{'title': 'pyML.ipynb', 'id': '1OvDhT-diTy51QpzqbRJEdr-do5Ne8QK3'}, {'id': '19s7dmpQxRUWCuoIQL-rgauzpmRQhaqed', 'title': 'seq2seq.krs', 'list': [{'title': 'saved_model.pb', 'id': '1--CqtL0rhwYNz27FA6w28MSlD2cFy4_Z'}, {'id': '1-1Eyueh03K35IaaqKdDvr9DveWL65hW6', 'title': 'assets', 'list': []}, {'id': '1-3-12eddgTS8wG_yoNoWR1DSUSqBKA7c', 'title': 'variables', 'list': [{'title': 'variables.index', 'id': '1-4QOvpsfOu8oSfCDnnp8DxZDSURx4ZEC'}, {'title': 'variables.data-00000-of-00002', 'id': '1-6MHSSjQm2RCP2OEge_0IBVn32x2nmrS'}, {'title': 'variables.data-00001-of-00002', 'id': '1-5jYUWf6tLlPtYvkt7KosVRunplBe55y'}]}]}, {'title': 'CM.txt', 'id': '1QHq6p67jpE56AI5c1HPhJifTpSYaB9Gl'}, '/root/my_sample_data/CM.txt']


In [0]:
import re
import kss
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
import time
choseong_list = [char for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"]

In [6]:
f = open('../root/my_sample_data/CM.txt', encoding="utf-8")
i=0
gogo= 0
sentence_list = []
lowCount = 2
longCount = 100
while True:
    line = f.readline()
    if not line: break
    if "<!DOCTYPE" in line:
        gogo = 0
    if "<text>" in line:
        gogo = 1
    if "</text>" in line:
        gogo = 0
    if gogo == 0: continue
    if line != '\n':
        i=i+1
        line = line.replace(u'\u3000', u"")
        line = line.replace(u'\x80', u"")
        line = line.replace(u'\x84', u"")
        line = line.replace(u'\x88', u"")
        line = line.replace(u'\x8a', u"")
        line = line.replace(u'\x8b', u"")
        line = line.replace(u'\x8c', u"")
        line = line.replace(u'\x90', u"")
        line = line.replace(u'\x94', u"")
        line = line.replace(u'\x97', u"")
        line = line.replace(u'\x9a', u"")
        line = line.replace(u'\xa0', u"")
        line = line.replace(u'\x9e', u"")
        line = line.replace(u'\x9d', u"")
        line = line.replace(u'\x81', u"")
        line = line.replace(u'\x83', u"")
        line = line.replace(u'\x8e', u"")
        line = line.replace(u'\x91', u"")
        line = line.replace(u'\x96', u"")
        line = line.replace(u'\x98', u"")
        line = line.replace(u'\x9b', u"")
        line = line.replace(u'\x9c', u"")
        line = line.replace(u'¤', u"")
        line = line.replace(u'¬', u"")
        line = line.replace(u'¶', u"")
        line = re.sub('<trunc>.+?<\/trunc>', '', line, 0, re.S)
        line = re.sub('<.+?>', '', line, 0, re.S)
        line = re.sub('\(.+?\)', '', line, 0, re.S)
        line = re.sub('\[.+?\]', '', line, 0, re.S)
        line = re.sub('\{.+?\}', '', line, 0, re.S)
        line = re.sub('《.+?》', '', line, 0, re.S)
        line = re.sub('[-§¨©°³¸¼´·²～¯¥=+,#/ꮀ\?:;遮¡^$@*\"※~&%ㆍ!『』\‘|\(\)\[\]\<\>`\'…《》\{\}_「」±√]', '', line,0,re.S)
        line = line.lower()
        line = re.sub('[a-z]', '', line, 0, re.S)
        line = re.sub('[A-Z]', '', line, 0, re.S)
        line_split = kss.split_sentences(line)
        temp_line = []
        for l_sp in line_split:
            if len(l_sp) >lowCount and len(l_sp) <longCount:
                temp_line.append(l_sp)
        sentence_list+=temp_line
f.close()
print(len(sentence_list))
print(sentence_list[:5])

184921
['뭘 좀 올려야지.', '뭘 좀 올렸어.', '다시 돌려 앞으로.', '됐어.', '우리가족은 아빠 엄마 오빠 나.']


In [7]:
#print(sentence_list[:5])
word_list = []
target = []
for sentence in sentence_list:
    sen2word = ""
    for word in sentence:
        if re.match('[가-힣]',word) is not None :
            sen2word+=choseong_list[(int)((ord(word)-ord('가'))/(21*28))]
        else :
            sen2word+=word
    target.append(sentence+'\n')
    word_list.append(sen2word+'\n')
    
#print(target[:5])
#print(word_list[:5])
src_vocab=set()
for words in word_list: # 1줄씩 읽음
    for char in words: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in target:
    for char in line:
        tar_vocab.add(char)
        
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
#print(src_vocab_size)
#print(tar_vocab_size)
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab)
print(tar_vocab[10:20])
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
#print(src_to_index)
#print(tar_to_index)
encoder_input = []
for line in word_list: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X[:-1])
print(encoder_input[:5])
decoder_input = []
for line in word_list:
    temp_X = []
    for w in line:
      temp_X.append(src_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])
decoder_target = []
for line in target:
    temp_X = []
    for w in line:
        temp_X.append(tar_to_index[w])
    decoder_target.append(temp_X)
print(decoder_target[:5])

['\n', ' ', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ᄇ', 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
['7', '8', '9', 'ᄇ', '가', '각', '간', '갇', '갈', '갉']
[[21, 2, 27, 2, 26, 20, 26, 27, 3], [21, 2, 27, 2, 26, 20, 26, 3], [18, 24, 2, 18, 20, 2, 26, 26, 20, 3], [18, 26, 3], [26, 20, 15, 27, 26, 2, 26, 23, 2, 26, 21, 2, 26, 23, 2, 17, 3]]
[[21, 2, 27, 2, 26, 20, 26, 27, 3, 1], [21, 2, 27, 2, 26, 20, 26, 3, 1], [18, 24, 2, 18, 20, 2, 26, 26, 20, 3, 1], [18, 26, 3, 1], [26, 20, 15, 27, 26, 2, 26, 23, 2, 26, 21, 2, 26, 23, 2, 17, 3, 1]]
[[691, 2, 1263, 2, 1128, 551, 1068, 1298, 3, 1], [691, 2, 1263, 2, 1128, 558, 1083, 3, 1], [339, 966, 2, 390, 551, 2, 1059, 1186, 562, 3, 1], [397, 1083, 3, 1], [1154, 605, 15, 1260, 1188, 2, 1046, 804, 2, 1090, 613, 2, 1125, 804, 2, 231, 3, 1]]


In [8]:
max_src_len = max([len(line) for line in word_list])-1
max_tar_len = max([len(line) for line in target])
print(max_src_len)
print(max_tar_len)
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

encoder_inputer = encoder_input[:len(encoder_input)//5*4]
decoder_inputer = decoder_input[:len(decoder_input)//5*4]
decoder_targeter = decoder_target[:len(decoder_target)//5*4]

val_encoder_inputer = encoder_input[len(encoder_input)//5*4:]
val_decoder_inputer = decoder_input[len(decoder_input)//5*4:]
val_decoder_targeter = decoder_target[len(decoder_target)//5*4:]


99
100


In [0]:
def oneHot_en_in(index,batch_size=32):
  return to_categorical(encoder_inputer[index*batch_size:(index+1)*batch_size], num_classes=src_vocab_size)
def oneHot_de_in(index,batch_size=32):
  return to_categorical(decoder_inputer[index*batch_size:(index+1)*batch_size], num_classes=src_vocab_size)
def oneHot_de_out(index,batch_size=32):
  return to_categorical(decoder_targeter[index*batch_size:(index+1)*batch_size], num_classes=tar_vocab_size)
#encoder_input = to_categorical(encoder_inputer, num_classes=src_vocab_size)
#decoder_input = to_categorical(decoder_inputer, num_classes=src_vocab_size)
#decoder_target = to_categorical(decoder_targeter, num_classes=tar_vocab_size)

In [0]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=100, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

decoder_inputs = Input(shape=(None, src_vocab_size))
decoder_lstm = LSTM(units=100, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [0]:
def shake_list():
  random.shuffle(encoder_inputer)
  random.shuffle(decoder_inputer)
  random.shuffle(decoder_targeter)

In [0]:
epochSize = 50
bathSize = 32
for j in range(epochSize):#len(encoder_inputer)):
  print("epoch :",j)
  print("batch : ",end='')
  start = time.time()
  shake_list()
  for i in range(len(encoder_inputer)//bathSize):
    if(i%(len(encoder_inputer)//bathSize//100)==0):
      print(">",end='')
    #model.train_on_batch(x=[encoder_input[(i)*32:(i+1)*32],decoder_input[(i)*32:(i+1)*32]], y= decoder_target[(i)*32:(i+1)*32])
    model.train_on_batch(x=[oneHot_en_in(i,batch_size=bathSize),oneHot_de_in(i,batch_size=bathSize)], y= oneHot_de_out(i,batch_size=bathSize))
  print("걸린시간 :",time.time()-start)


epoch : 0
batch : >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>걸린시간 : 481.8747310638428
epoch : 1
batch : >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>걸린시간 : 474.6745607852936
epoch : 2
batch : >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [0]:
#model.fit(x=[encoder_input, decoder_input], y=decoder_target,  epochs=50, validation_split=0.2, batch_size=64)

In [0]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [0]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(100,))
decoder_state_input_c = Input(shape=(100,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [0]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [0]:
def str2cho(input_str):
    #문장을 입력받으면 초성 문자열로반환하는 코드
    cho = ""
    for word in input_str:
        if re.match('[가-힣]',word) is not None :
            cho+=(choseong_list[(int)((ord(word)-ord('가'))/(21*28))])
        else :
            cho+=(word)
    return cho

In [0]:
def decode_sequence(input_seq,strWORD):
    # 입력으로부터 인코더의 상태를 얻음
    input_seq = to_categorical(input_seq, num_classes=src_vocab_size)
    states_value = encoder_model.predict(input_seq)
    # 첫글자에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, src_vocab_size))
    target_seq[0, 0, src_to_index[str2cho(strWORD[0])]] = 1.

    stop_condition = False
    decoded_sentence = ""
    cho_sen = ""
    cho_sen+=str2cho(strWORD[0])
    itr = 0
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 입력 글자를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) >= len(strWORD)):
            stop_condition = True
        else:
          itr+=1

        # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
        target_seq = np.zeros((1, 1, src_vocab_size))
        
        target_seq[0, 0, src_to_index[str2cho((strWORD[itr]))]] = 1.
        cho_sen+=str2cho(strWORD[itr])
        # 상태를 업데이트 합니다.
        states_value = [h, c]
    #print(cho_sen)
    return decoded_sentence

In [0]:
i = 0
for seq_index in [1,2,16000,4,5]:#105,70000,15002,40015,5]: # 입력 문장의 인덱스
    i+=1
    print('입력 문장:', word_list[seq_index])
    print('정답 문장:', target[seq_index][:len(target[seq_index])-1]) # '\t'와 '\n'을 빼고 출력

    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,word_list[seq_index])
    #print('입력 문장:', word_list[seq_index])
    #print('정답 문장:', target[seq_index][:len(target[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)]) # '\n'을 빼고 출력
    print(35 * "-")

In [0]:
model.summary()
from os import path
from google.colab import drive

notebooks_dir_name = 'MLTEXT'
drive.mount('/content/gdrive')
notebooks_base_dir = path.join('./gdrive/My Drive/', notebooks_dir_name)
if not path.exists(notebooks_base_dir):
  print('Check your google drive directory. See you file explorer')

model.save(path.join(notebooks_base_dir, "seq2seq.krs"))